# 🎓 Theory Primer: Quantum Kernel Methods

**A Gentle Introduction for Researchers New to Quantum Machine Learning**

---

This notebook provides the theoretical background needed to understand the experiments in this repository. We'll cover:

1. What are kernel methods?
2. Classical vs Quantum kernels
3. Feature maps and Hilbert spaces
4. Why eigenvalues measure expressivity

No quantum computing experience required! 🚀

## 1. What Are Kernel Methods?

### The Problem
Many machine learning problems are **not linearly separable**. For example, consider points arranged in concentric circles - no straight line can separate them.

### The Solution: Feature Maps
A **feature map** $\Phi$ transforms data into a higher-dimensional space where it *becomes* linearly separable:

$$x \in \mathbb{R}^d \xrightarrow{\Phi} \Phi(x) \in \mathbb{R}^D$$

where $D \gg d$ (many more dimensions).

### The Kernel Trick
Computing $\Phi(x)$ explicitly can be expensive. The **kernel trick** lets us compute similarity in the high-dimensional space *without* ever computing $\Phi$ directly:

$$K(x, y) = \langle \Phi(x), \Phi(y) \rangle$$

This inner product is all we need for algorithms like SVM!

In [ ]:
# Let's visualize the concentric circles problem
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

# Generate concentric circles
X, y = make_circles(n_samples=100, factor=0.5, noise=0.05, random_state=42)

plt.figure(figsize=(8, 6))
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Class 0', s=50)
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Class 1', s=50)
plt.title('Concentric Circles: NOT Linearly Separable', fontsize=14)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print("❓ Can you draw a single straight line to separate blue from red?")
print("   Answer: No! That's why we need kernel methods.")

## 2. The RBF (Radial Basis Function) Kernel

The most popular classical kernel is the **RBF kernel** (also called Gaussian kernel):

$$K_{RBF}(x, y) = \exp\left(-\gamma \|x - y\|^2\right)$$

### Key Properties:
- **Distance-based**: Similarity depends on Euclidean distance
- **Local**: Nearby points have high similarity, distant points have low similarity
- **Smooth decay**: Similarity decreases smoothly with distance

### The γ (gamma) parameter:
- Small γ → Wide kernel, more points considered "similar"
- Large γ → Narrow kernel, only very close points are similar

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel

# Compute RBF kernel on our circles data
K_rbf = rbf_kernel(X, gamma=1.0)

# Visualize the kernel matrix
plt.figure(figsize=(10, 8))
plt.imshow(K_rbf, cmap='viridis')
plt.colorbar(label='Similarity')
plt.title('RBF Kernel Matrix\n(Diagonal = self-similarity = 1.0)', fontsize=14)
plt.xlabel('Sample Index')
plt.ylabel('Sample Index')
plt.show()

print("🔍 Notice the diagonal blur pattern.")
print("   This is because RBF is 'myopic' - it only sees local neighborhoods.")

## 3. Quantum Kernels: A Different Geometry

### The Quantum Approach
Instead of mapping to $\mathbb{R}^D$, quantum kernels map data to **Hilbert space** using quantum circuits:

$$x \xrightarrow{U(x)} |\Phi(x)\rangle \in \mathbb{C}^{2^n}$$

Where:
- $U(x)$ is a parameterized quantum circuit
- $n$ is the number of qubits
- $2^n$ is the Hilbert space dimension (exponentially large!)

### Quantum Kernel (Fidelity)
The quantum kernel measures the **overlap** between quantum states:

$$K_Q(x, y) = |\langle \Phi(x) | \Phi(y) \rangle|^2$$

### The Key Difference: ENTANGLEMENT
- Classical: Each feature is processed independently
- Quantum: Entanglement creates **correlations** between features

This allows quantum kernels to capture patterns invisible to classical methods!

In [ ]:
# Let's visualize a quantum feature map circuit
from qiskit.circuit.library import ZZFeatureMap

# Create a 2-qubit ZZFeatureMap
feature_map = ZZFeatureMap(feature_dimension=2, reps=1, entanglement='linear')

print("🔮 ZZ Feature Map Circuit (2 qubits):")
print("="*50)
print(feature_map.decompose())
print("\n📌 Key elements:")
print("   - H gates: Create superposition")
print("   - P gates: Encode data as rotation angles")
print("   - CX gates: Create ENTANGLEMENT between qubits")

## 4. Why Eigenvalues Measure Expressivity

### Eigenvalue Decomposition
Every kernel matrix $K$ can be decomposed as:

$$K = V \Lambda V^T$$

Where:
- $V$ = eigenvectors (directions in feature space)
- $\Lambda$ = eigenvalues $\lambda_1 \geq \lambda_2 \geq ... \geq \lambda_n \geq 0$

### What Eigenvalues Tell Us
The **eigenvalue spectrum** reveals the "effective dimensionality":

| Pattern | Meaning |
|:--------|:--------|
| Fast decay (λ → 0 quickly) | Low rank, compressed representation |
| Slow decay (λ stays high) | High rank, expressive representation |

### The Quantum Advantage
If quantum eigenvalues decay slower than classical, the quantum kernel accesses **more dimensions** of the feature space — higher expressivity!

In [ ]:
# Visualize eigenvalue decay
eigenvalues = np.linalg.eigvalsh(K_rbf)[::-1]  # Sort descending
eigenvalues = eigenvalues / eigenvalues[0]     # Normalize

plt.figure(figsize=(10, 6))
plt.semilogy(eigenvalues, 'o-', color='purple')
plt.axhline(y=0.01, color='red', linestyle='--', label='1% threshold')
plt.fill_between(range(len(eigenvalues)), eigenvalues, 0.01, 
                 where=eigenvalues > 0.01, alpha=0.3, color='purple')
plt.title('Eigenvalue Spectrum of RBF Kernel', fontsize=14)
plt.xlabel('Component Index')
plt.ylabel('Eigenvalue (log scale)')
plt.legend()
plt.grid(True, alpha=0.5)
plt.show()

effective_rank = np.sum(eigenvalues > 0.01)
print(f"📊 Effective Rank (λ > 1%): {effective_rank} out of {len(eigenvalues)}")
print(f"   This means DaRBF kernel uses {effective_rank} significant dimensions.")

## 5. Key Takeaways

| Concept | Classical (RBF) | Quantum (Fidelity) |
|:--------|:----------------|:-------------------|
| **Similarity Measure** | Euclidean distance | Hilbert space overlap |
| **Geometry** | Local (myopic) | Non-local (interference) |
| **Feature Space** | Infinite-dim RKHS | $2^n$ dimensional |
| **Correlations** | None | Entanglement-induced |

---

## 🚀 Next Steps

Now that you understand the theory, explore the experiments:

1. **Notebook 02**: Visual comparison of kernel heatmaps ("Fog vs Crystal")
2. **Notebook 03**: Eigenvalue analysis proving expressivity differences
3. **Notebook 04**: Scaling experiment showing "Rank Explosion" at 8 qubits

Happy exploring! 🔬